In [6]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
LOCAL_DATA_PATH='data'
STREAMING_TRACES=['youtube', 'bilibili']
PROTOCOLS=['udp', 'tcp']
FEATURE_LEVEL=['flow', 'pkt']
FEATURE_FILE_TYPE = 'csv'